<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/historical_surface_water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Inland Surface Water Area

In [168]:
import geemap
import ee

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

Used this tool to create a geojson file of the area of interest:

[keene Polyline Tool](https://www.keene.edu/campus/maps/tool/?coordinates=77.1200409%2C%2011.5324541%0A76.9923248%2C%2011.5062217%0A76.9916382%2C%2011.3467571%0A77.1529998%2C%2011.4261642%0A77.1200409%2C%2011.5324541) : Use this tool to create a polygon and generate a GeoJson text for further use

In [169]:
geojson  = {
  "coordinates": [[[73.777309817,15.6646429326],[73.9436840308,15.6646429326],[73.9436840308,15.7947142468],[73.777309817,15.7947142468],[73.777309817,15.6646429326]]],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Define Functions

In [170]:
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

def addNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    water_mask = ndwi.gte(0).rename('water_mask')
    return image.addBands(ndwi).addBands(water_mask)

## Downloading the Global Surface Water data


The `maskS2clouds` function ensures that only clear-sky pixels are retained in the image, which is crucial for accurate analysis and visualization

In [192]:
# Define the date range for the image collection.
year = 2015
start_date = f'{year}-01-01'
end_date = f'{year}-12-31'

# Load the Sentinel-2 image collection.
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .map(maskS2clouds) \
            .map(addNDWI) \
            .filter(ee.Filter.bounds(geometry))

In [195]:
img_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(ee.Filter.bounds(geometry)).filterDate(start_date, end_date)
img_col.size()

In [193]:
dataset.size()

In [179]:
Map = geemap.Map()

water_image = dataset.median().select('water_mask').selfMask()

vizParams = {
    'max': 1,
    'palette': ['#0561f5']
}

# Define visualization parameters for RGB
rgb_vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3,
}

# Add the Sentinel-2 image to the map
Map.addLayer(dataset.first(), rgb_vis_params, 'Sentinel-2 RGB')

# Add the bounding box to the map with an outline and no fill
Map.add_basemap("HYBRID")
Map.addLayer(geometry, {'color': 'blue', "fill": False, 'opacity': 0.2}, 'Bounding Box')
Map.addLayer(water_image.clip(geometry), vizParams, 'NDWI')
Map.centerObject(geometry, 12)

Map

EEException: Image.visualize: Parameter 'image' is required.

## Water Index using Landsat Satellite data

In [242]:
def filterImageCollection(
        image_collection_id: str,
        geometry: ee.geometry.Geometry,
        date_range: tuple[str],
        cloud_cover = 10
    ) -> ee.imagecollection.ImageCollection:

    imageCollection = ee.ImageCollection(image_collection_id)
    d1,d2 = date_range
    filtered = imageCollection.filter(
       ee.Filter.lt('CLOUD_COVER', 10)
    )\
    .filter(
        ee.Filter.date(d1, d2) ## Filter by date
    ).filter(
        ee.Filter.bounds(geometry) ## Filter by bounds
    )

    return filtered

def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [243]:
y1,y2 = (1984, 2025)
cloud_cover = 10
date_range = (f"{y1}-01-01", f"{y2}-01-01")

## get the landsat 5 image collection
landsat5 = filterImageCollection(
    image_collection_id = "LANDSAT/LT05/C02/T1_L2",
    geometry = geometry,
    date_range = date_range,
    cloud_cover = cloud_cover
)


## get the landsat 7 image collection
landsat7 = filterImageCollection(
    image_collection_id = "LANDSAT/LC08/C02/T1_L2",
    geometry = geometry,
    date_range = date_range,
    cloud_cover = cloud_cover
)

## get the landsat 8 image collection
landsat8 = filterImageCollection(
    image_collection_id = "LANDSAT/LC08/C02/T1_L2",
    geometry = geometry,
    date_range = date_range,
    cloud_cover = cloud_cover
)

## get the landsat 8 image collection
landsat9 = filterImageCollection(
    image_collection_id = "LANDSAT/LC09/C02/T1_L2",
    geometry = geometry,
    date_range = date_range,
    cloud_cover = cloud_cover
)

## merge the image collection
## Sort them as per time
landsat = landsat8.merge(landsat9).merge(landsat7).merge(landsat5).sort('system:time_start')
landsat = landsat.map(apply_scale_factors)

In [244]:
landsat.size()

In [245]:
def preprocess_landsat(image:ee.image.Image) -> ee.image.Image:
    '''
    This function will choose only the required bands
    also rescale the reflectance value to original
    and calculate the indices: NDVI
    '''

    ## Calculate the NDWI
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
    water_mask = ndwi.gte(0).rename('water_mask')

    return image.addBands(water_mask).addBands(ndwi)


ndwi_landsat = landsat.map(preprocess_landsat)

In [246]:
year = 1995
start_date = f'{year}-01-01'
end_date = f'{year+1}-01-01'

images = ndwi_landsat.filterDate(start_date, end_date)
mosaic_image = images.mosaic()

In [247]:
Map = geemap.Map()

water_image = images.median().select('water_mask').selfMask()

vizParams = {
    'max': 1,
    'palette': ['#0561f5']
}

# Define visualization parameters for RGB
rgb_vis_params = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0,
    'max': 0.3,
}

# Add the Sentinel-2 image to the map
Map.addLayer(mosaic_image, rgb_vis_params, 'Landsat')

# Add the bounding box to the map with an outline and no fill
Map.add_basemap("HYBRID")
Map.addLayer(geometry, {'color': 'blue', "fill": False, 'opacity': 0.2}, 'Bounding Box')
Map.addLayer(water_image.clip(geometry), vizParams, 'NDWI')
Map.centerObject(geometry, 12)

Map

Map(center=[15.729687417562605, 73.86049692390029], controls=(WidgetControl(options=['position', 'transparent_…